In [6]:
from glob import glob
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import plotly.graph_objs as go
from mooringParams import paramsDAFT1 as processParams # Each mooring has a specific params file with 
from scipy.interpolate import splprep,splev
import math
compassCor = {1:335,2:208,3:250,4:197,5:293}

In [2]:
dfs = []
for file in tqdm(glob(processParams.evParams.outputDir+'exports\\'+'*(regions).csv')):
    dfs.append(pd.read_csv(file))
dfR = pd.concat(dfs)
dfs = []

for file in tqdm(glob(processParams.evParams.outputDir+'exports\\'+'*(targets).csv')):
    dfs.append(pd.read_csv(file))
dfT = pd.concat(dfs)
cols = np.append(dfT.columns[0:18],['Ping_number']).tolist()
dfT = dfT[cols].drop(columns=['Region_name', 'Region_class'])
dfR = dfR.drop(columns=['Region_name', 'Region_class','Target_length_mean'])
dfR = dfR[dfR.TS_max < -30]
dfT = dfT[dfT.Region_ID.isin(dfR.Region_ID)]

In [21]:
# Using Echoview terminology, so Major = X, Minor = Y
def targetSpline(df, processID, regionID,path_direct=False):
    df = df[(df.Region_ID==regionID) & (df.Process_ID==processID)]
    tck, u  = splprep([df.Distance_major_axis, df.Distance_minor_axis,df.Target_true_depth],k=3)
    if path_direct:
        x_knots, y_knots, z_knots = splev([0,1], tck)
    else:
        x_knots, y_knots, z_knots = splev(np.linspace(0,1,len(df.Distance_major_axis)), tck)
    return  x_knots, y_knots, z_knots

def trackPlot(df,processID, regionID, plotElements =['track_smoothed','track','track_direct','targets']):
    # plotElements: list of elements to include:
    # targets: target locations
    # track: line between targets
    # track_smoothed: Cubic spline with equal resolution to the number of targets
    # track_direct: "flux" line from start to end target based on spline    
    x_knots, y_knots, z_knots = targetSpline(df, processID, regionID ,path_direct=False)
    track_smoothed = go.Scatter3d(x=x_knots, y=y_knots, z=z_knots,mode='lines',name='Smoothed Line')
    
    x_knots, y_knots, z_knots = targetSpline(df, processID, regionID ,path_direct=True)
    track_direct = go.Scatter3d(x=x_knots, y=y_knots, z=z_knots,mode='lines',name='Direct Path')

    dfCur = df[(df.Region_ID==regionID) & (df.Process_ID==processID)]
    targets=go.Scatter3d(x=dfCur.Distance_major_axis, y=dfCur.Distance_minor_axis, z=dfCur.Target_true_depth,mode='markers',
        marker=dict(size=8,color='black', colorscale='plasma',opacity=0.8,colorbar=dict(title='TS',thickness=30),showscale=False),name='Targets')
    track = go.Scatter3d(x=dfCur.Distance_major_axis, y=dfCur.Distance_minor_axis, z=dfCur.Target_true_depth,mode='lines',name='Target Line')

    allElements = {'track_smoothed':track_smoothed,'track':track,'track_direct':track_direct,'targets':targets}
    fig = go.Figure(data=[allElements[i] for i in plotElements])
    split = (np.ceil(np.max([dfCur.Distance_major_axis.max() - dfCur.Distance_major_axis.min(),dfCur.Distance_minor_axis.max() - dfCur.Distance_minor_axis.min(),dfCur.Target_true_depth.max() - dfCur.Target_true_depth.min()]))/2)+.25
    fig.update_layout(showlegend=True,scene = dict(
                        xaxis_title='Distance_major_axis',
                        yaxis_title='Distance_minor_axis',
                        zaxis_title='Depth'),
                        #xaxis = dict(nticks=4, range=[np.median(dfCur.Distance_major_axis)-split, np.median(dfCur.Distance_major_axis)+split]),
                        #yaxis = dict(nticks=4, range=[np.median(dfCur.Distance_minor_axis)-split, np.median(dfCur.Distance_minor_axis)+split]),
                       # zaxis = dict(nticks=4, range=[np.median(dfCur.Target_true_depth)-split, np.median(dfCur.Target_true_depth)+split])),
                        width=700,
                        margin=dict(r=20, b=10, l=10, t=10))

    fig.show()

    
def regionStats(dfR,dfT,compassOffset,path_direct=False):
    x_knots, y_knots, z_knots = targetSpline(dfT, dfR.Process_ID, dfR.Region_ID ,path_direct=path_direct)
    coords = [j for j in zip(x_knots,y_knots,z_knots)]
    d = np.diff(coords, axis=0)
    segdists = np.hypot(d[:,0], d[:,1])
    tSpeed=segdists.sum()/((dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.max() - dfT[(dfT.Process_ID == dfR.Process_ID)& (dfT.Region_ID == dfR.Region_ID)].Ping_number.min())*.4)
    if path_direct:
        tDir = direction_lookup(compassOffset,x_knots[1],x_knots[0],y_knots[1],y_knots[0])
    else:
        a = []
        for i in range(len(x_knots)-1):
            a.append(direction_lookup(compassOffset,x_knots[i+1],x_knots[i],y_knots[i+1],y_knots[i]))
        tDir = np.mean(a)
    return tSpeed, segdists.sum(), tDir

def direction_lookup(compassOffset,destination_x, origin_x, destination_y, origin_y):
    deltaX = destination_x - origin_x
    deltaY = destination_y - origin_y
    degrees_temp = math.atan2(deltaX, deltaY)/math.pi*180
    # Flip for the inversion due to upward facing Xducer
    degrees_temp = 360-degrees_temp
    # Correct for the Xducer direction
    degrees_temp = (degrees_temp + compassOffset-360)
    # Convert check if its negative and correct
    if degrees_temp < 0:
        degrees_final = 360 + degrees_temp
    if degrees_temp > 360:
        degrees_final = degrees_temp-360
    else:
        degrees_final = degrees_temp
    return degrees_final

In [8]:
i = 50
trackPlot(dfT,dfR.iloc[i].Process_ID, dfR.iloc[i].Region_ID)
regionStats(dfR.iloc[i],dfT,compassCor[processParams.wbatParams.mooringNum],path_direct=False)

(0.297082492242037, 222.94892160440287)

In [30]:
ss,ds = [],[]
for index,row in tqdm(dfR.iterrows(),total=len(dfR)):
    s,dist,d = regionStats(row,dfT,compassCor[processParams.wbatParams.mooringNum],path_direct=True)
    ss.append(s)
    ds.append(d)

In [ ]:
dfR['splineDir'][dfR['splineDir'] >360] = dfR['splineDir'][dfR['splineDir'] >360]-360
import matplotlib.pyplot as plt
plt.hist(dfR.splineDir,90)